In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import re
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
import os

In [3]:
## Normal dataset
#df = pd.read_csv("CSVFiles/smallDomainDataNotEmbedded.csv")

#df = pd.read_csv("CSVFiles/smallDomainDataBertweetWordEmbeds.csv")


import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

df = pd.read_csv("CSVFiles/smallDomainDataNotEmbedded.csv")


bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)                     
bertweet.eval()
bertweet.to('cuda')

with torch.no_grad():
    for i, idx in enumerate(df.index):
        embeds = bertweet(torch.tensor(tokenizer.encode(df['comment_text'][idx], add_special_tokens=True,
                                                                   truncation=True)).to('cuda').unsqueeze(0))[0].squeeze(0)
        if embeds.shape[0] < 100:
            embeds = torch.cat((embeds, torch.zeros((100-embeds.shape[0], 768)).to('cuda')))
            
        elif embeds.shape[0] > 100:
            embeds = embeds[0:100]
        
        df.at[idx, 'comment_text'] = embeds.detach().cpu().numpy()

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Updating values for training_data
training_data = df[df['split'] == 'train']
#training_data = training_data.drop(training_data.query('toxicity==0').sample(frac=.85).index)

# Getting test_data
test_data = df[df['split'] == 'test']

# Getting validation_data
validation_data = df[df['split'] == 'val']
#validation_data = validation_data.drop(validation_data.query('toxicity==0').sample(frac=.85).index)

In [5]:
# Creating data loaders
X_train = np.array(training_data['comment_text'].values.tolist())
Y_train = np.array(training_data['toxicity'].values.tolist())

X_test = np.array(test_data['comment_text'].values.tolist())
Y_test = np.array(test_data['toxicity'].values.tolist())

X_val = np.array(validation_data['comment_text'].values.tolist())
Y_val = np.array(validation_data['toxicity'].values.tolist())

In [6]:
print(len(X_train))
print(len(X_test))
print(len(X_train))
print(len(X_val))

1420
720
1420
720


In [7]:
def CleanText(text):
    text = text.lower() #Turn all text entries into lower-case
    text = re.sub(r'''(https?:\/\/www\.|https?:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,3}[-a-zA-Z0-9()@:%_\+.~#?&\//=<>]*''', "<URL>", text)
    #Replace URL with tag
    text = re.sub(r'''[0-9]+[/\-.]+[0-9]+[/\-.]+[0-9]+''', "<DATE>", text) #Replace dates with tag
    text = re.sub(r'''[a-z0-9._%+-]+\@[a-z0-9.-]+[a-z0-9]\.[a-z]{1,}''', "<EMAIL>", text)
    text = re.sub(r'''[0-9]+''', "<NUM>", text) #Replace numbers with tag
    text = re.sub(r'''[.|,|!|?|\'|\''|\"|\n|\t|\-|\(|\)]''', '', text)
    text = re.sub(r'''^\s+|\s+$''', '', text) #Remove whitespaces at the end and start of string
    text = re.sub(r'''[ ][ ]+|_''', " ", text) #Remove multiple whitespace
    return text

In [20]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#model = AutoModel.from_pretrained("bert-base-uncased")
#model.eval()
#model.to('cuda')

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
model.to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [9]:
prep_training = []
prep_test = []
prep_val = []

for i, text in enumerate(X_train):
    prep_training.append([torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length=100, truncation=True, padding="max_length")), Y_train[i]])

for i, text in enumerate(X_test):
    prep_test.append([torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length=100, truncation=True, padding="max_length")), Y_test[i]])

for i, text in enumerate(X_val):
    prep_val.append([torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length=100, truncation=True, padding="max_length")), Y_val[i]])

In [10]:
train_loader = DataLoader(prep_training, batch_size=16, shuffle=True)
valid_loader = DataLoader(prep_val, batch_size=16, shuffle=False)
test_loader = DataLoader(prep_test, batch_size=16, shuffle=False)

In [7]:
def CheckAccuracy(predictions, labels):
        acc = 0.0
        for i in range(len(predictions)):
            if (predictions[i] == labels[i]):
                acc += 1
        return acc/len(predictions)

In [8]:
epochs = 50
lr = 0.00005

cuda = True # Set this if training on GPU
cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [9]:
prep_training = []
prep_test = []
prep_val = []

for i, text in enumerate(X_train):
    prep_training.append([X_train[i], Y_train[i]])

for i, text in enumerate(X_test):
    prep_test.append([X_test[i], Y_test[i]])

for i, text in enumerate(X_val):
    prep_val.append([X_val[i], Y_val[i]])
    
train_loader = DataLoader(prep_training, batch_size=16, shuffle=True)
valid_loader = DataLoader(prep_val, batch_size=16, shuffle=False)
test_loader = DataLoader(prep_test, batch_size=16, shuffle=False)

In [10]:

# [batch_size, seq_len, input_size]
class LSTM_Net(nn.Module):
    
    def __init__(self):
        super(LSTM_Net, self).__init__()
        self.lstm = nn.LSTM(input_size=768, hidden_size=768, num_layers=1, batch_first=True, bidirectional=True) #bidirectional=True
        self.fc1 = nn.Sequential(nn.Linear(768, 256), nn.ReLU(), nn.Dropout(p=0.8))
        self.fc2 = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #x = model(x)[0]
        lstm_out, (ht, ct) = self.lstm(x)
        #print(lstm_out.shape)
        #print(ht.shape)
        #print(ht[-1, :, :].shape)
        output = self.fc1(ht[-1, :, :])
        output = self.fc2(ht[-1, :, :])
        #print(output.shape)
        output = self.sigmoid(output)
        return output

In [11]:
# Setting up model parameters
lstm_model = LSTM_Net().to(device)
loss_function = nn.BCELoss()
optimizer = torch.optim.AdamW(lstm_model.parameters(), lr=lr)

early_stopping = 5
notImproved = 0
bestLoss = None
bestModel = None
bestEpoch = 0

trainArr = []
valArr = []

for epoch in range(1, epochs + 1): 
    
    train_loss = 0.0
    lstm_model.train()
    for batch_idx, data in enumerate(train_loader):
        
        # get the input
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        
        optimizer.zero_grad()        
        outputs = lstm_model(inputs)
        
        outputs = outputs.squeeze(1)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader.dataset)
    
    trainArr.append(train_loss)
    
    #print("train loss: ", train_loss)
    
    valid_loss = 0
    labs = []
    preds = []
    
    lstm_model.eval()
    with torch.no_grad():        
        for batch_idx, data in enumerate(valid_loader):
            # get the input
            inputs, labels = data
        
            inputs = inputs.to(device)
            labels = labels.to(device).float()
            
            outputs = lstm_model(inputs)
            outputs = outputs.squeeze(1)
        
            labs.extend(labels)
            preds.extend(torch.round(outputs))
            valid_loss += loss_function(outputs, labels).item()
            
    valid_loss /= len(valid_loader.dataset)
    
    valArr.append(valid_loss)
    
    #print("valid loss: ", valid_loss)
    print("Accuracy on validation set: ", CheckAccuracy(labs, preds))
    
    if bestLoss == None:
        bestLoss = valid_loss
    
    if valid_loss <= bestLoss:
        bestLoss = valid_loss
        bestModel = lstm_model
        notImproved = 0
        bestEpoch = epoch
    else:
        notImproved +=1
        
    if notImproved >= early_stopping:
        break
    
print(bestEpoch)

lstm_model = bestModel

Accuracy on validation set:  0.5375
Accuracy on validation set:  0.5486111111111112
Accuracy on validation set:  0.5708333333333333
Accuracy on validation set:  0.5944444444444444
Accuracy on validation set:  0.6041666666666666
Accuracy on validation set:  0.5833333333333334
Accuracy on validation set:  0.5888888888888889


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(np.arange(0,len(trainArr)), trainArr, color='r', label='Training loss')
plt.scatter(np.arange(0,len(valArr)), valArr, color='g', label='Validation loss')
plt.title("Training loss vs Validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()

In [18]:
def F1_Scores(preds, labels):
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    for i in range(len(labs)):
        if labels[i]==1 and preds[i]==1:
            true_positives += 1
        if labels[i]==0 and preds[i]==0:
            true_negatives += 1
        if labels[i]==0 and preds[i]==1:
            false_positives += 1
        if labels[i]==1 and preds[i]==0:
            false_negatives += 1
    print("true_positives", true_positives)
    print("true_negatives", true_negatives)
    print("false_positives", false_positives)
    print("false_negatives", false_negatives)
    
    return true_positives, true_negatives, false_positives, false_negatives

In [20]:
labs = []
preds = []
lstm_model.eval()
with torch.no_grad():        
    for batch_idx, data in enumerate(test_loader):
        # get the input
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        
        # forward + backward + optimize
        outputs = lstm_model(inputs)
        outputs = outputs.squeeze(1)
        
        labs.extend(labels)
        preds.extend(torch.round(outputs))
print("Accuracy on test set: ", CheckAccuracy(labs, preds))

labs = []
preds = []
lstm_model.eval()
with torch.no_grad():        
    for batch_idx, data in enumerate(valid_loader):
        # get the input
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        
        # forward + backward + optimize
        outputs = lstm_model(inputs)
        outputs = outputs.squeeze(1)
        
        labs.extend(labels)
        preds.extend(torch.round(outputs))
print("Accuracy on val set: ", CheckAccuracy(labs, preds))

labs = []
preds = []
lstm_model.eval()
with torch.no_grad():        
    for batch_idx, data in enumerate(train_loader):
        # get the input
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        
        # forward + backward + optimize
        outputs = lstm_model(inputs)
        outputs = outputs.squeeze(1)
        
        labs.extend(labels)
        preds.extend(torch.round(outputs))
print("Accuracy on train set: ", CheckAccuracy(labs, preds))

Accuracy on test set:  0.6569444444444444
Accuracy on val set:  0.6555555555555556
Accuracy on train set:  0.9845070422535211
